**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act takes in a state and returns the action to perform by the agent. The parameter `epsilon` is used for exploration: with probability $1-\epsilon$, the agent will choose the action that he predicts as the best for the state he is currently in, and with probability $\epsilon$ he will choose a random action instead. This allows the agent to explore the action space and try out different options.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=101 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array `position` is an array of shape `(grid_size x grid_size)` which is equal to 1 where the rat currently is and equal to 0 everywhere else except at the borders of the map where it is equal to -1. This array is used to tell the neural network where the rat actually is. 

The array `board` is an array of shape `(grid_size x grid_size)` which represents the reward of each cell on the map. It is equal to 0.5 if there is a malus on the cell, -1 if there is a bonus and 0 otherwise. When the rat visits a cell, the reward is removed and the value of the board at this position is set to 0. This array is used to tell the neural network where and what the rewards are on the map.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
         # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        
        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/24.0. Average score (-14.5)
Win/lose count 10.0/19.0. Average score (-11.75)
Win/lose count 8.0/7.0. Average score (-7.5)
Win/lose count 7.5/18.0. Average score (-8.25)
Win/lose count 10.5/17.0. Average score (-7.9)
Win/lose count 10.5/5.0. Average score (-5.666666666666667)
Win/lose count 9.0/16.0. Average score (-5.857142857142857)
Win/lose count 6.0/13.0. Average score (-6.0)
Win/lose count 9.5/13.0. Average score (-5.722222222222222)
Win/lose count 8.5/13.0. Average score (-5.6)
Win/lose count 14.5/13.0. Average score (-4.954545454545454)
Win/lose count 12.0/16.0. Average score (-4.875)
Win/lose count 8.5/13.0. Average score (-4.846153846153846)
Win/lose count 5.5/8.0. Average score (-4.678571428571429)
Win/lose count 10.0/12.0. Average score (-4.5)
Win/lose count 8.5/20.0. Average score (-4.9375)
Win/lose count 8.5/10.0. Average score (-4.735294117647059)
Win/lose count 6.0/10.0. Average score (-4.694444444444445)
Win/lose count 9.5/14.0. Average score (-4.68421

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




\begin{equation}
\begin{array}{lcl}
Q^\pi(s,a)&=&E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
    & =& \mathbb{E}_{P^{\pi}} [r(s,a) +  \sum_{t\geq1}^{\infty} \gamma^{t} r(s_t, a_t) | s_0=s, a_0=a ] \\
    & = & r(s,a) + \gamma \mathbb{E}_{P^{\pi}} [\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a ]\\
    &=&  r(s,a) + \gamma\sum_{(a',s')}  \mathbb{E}_{P^{\pi}} [\sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a, s_1=s', a_1=a' ] \times \mathbb{P}(s_1=s', a_1=a'|s_0=s, a_0=a)\\
    &=& r(s,a) + \gamma \sum_{(a',s')}  \mathbb{E}_{P^{\pi}} [\sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_1=s', a_1=a' ] \times \mathbb{P}(s_1=s', a_1=a'|s_0=s, a_0=a) \\
    &=& r(s,a) + \gamma \sum_{(a',s')} \mathbb{P}(s_1=s', a_1=a'|s_0=s, a_0=a) \times  Q^{\pi}(s', a') \\
    &=& r(s,a) + \gamma \mathbb{E}_{(s',a')\sim P(\cdot|s,a)} [ Q^{\pi}(s', a') ] \\
    &=& \mathbb{E}_{(s',a')\sim P(\cdot|s,a)} [ r(s,a) + \gamma Q^{\pi}(s', a') ]
\end{array}
\end{equation}





We have:
\begin{align*} Q^*(s,a) &= \max_{\pi}Q^{\pi}(s,a) \\
&=\max_{\pi}[r(s,a) + \gamma \mathbb{E}_{(s',a')}[Q^{\pi}(s',a')]] \\
&=  r(s,a) + \gamma \max_{\pi}[\mathbb{E}_{(s',a')}[Q^{\pi}(s',a')]] \\
\end{align*}
We need to show that:
\begin{equation*} \max_{\pi}\mathbb{E}_{(s',a')\sim p(\cdot|s,a)}[Q^{\pi}(s',a')]=\mathbb{E}_{s'' \sim \pi^*(\cdot|s,a)}[\max_{a''}Q^*(s'',a'')]
\end{equation*}

We have : $$\max \limits_{\pi} Q^{\pi}(s', a') = \max \limits_{a''} \max \limits_{\pi'} Q^{\pi'}(s', a'') = \max \limits_{a''} Q^{*}(s', a'')$$
where $\pi'$ is a policy starting from the second step.

So on one hand we have:
\begin{align*}  \max_{\pi}\mathbb{E}_{(s',a')\sim p(\cdot|s,a)}[Q^{\pi}(s',a')] 
&\leq \mathbb{E}_{s'' \sim \pi^*(\cdot|s,a)}[\max_{a''}Q^{*}(s',a'')]
\end{align*}


And on the other hand:

\begin{align*}\mathbb{E}_{s'' \sim \pi^*(\cdot|s,a)}[\max_{a''}Q^*(s',a'')] &= \max_{a''}\mathbb{E}_{s'' \sim \pi^*(\cdot|s,a)}[Q^{\pi^*}(s',a'')] \\
&\leq \max_{a''}\max_{\pi}\mathbb{E}_{(s',a')\sim p^{\pi}(\cdot|s,a)}[Q^{\pi}(s',a')] \\
&\leq \max_{\pi}\mathbb{E}_{(s',a')\sim p^{\pi}(\cdot|s,a)}[Q^{\pi}(s',a')] 
\end{align*}

Which gives us the result:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Our goal is for $Q(s,a,\theta)$ to be as close as possible to $Q^{*}(s,a)$ which we can approximate by $E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)\Vert$.
Therefore, a plausible objective is:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) <= self.max_memory:
            self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            target_q[i] = self.model.predict(np.expand_dims(s_, 0))
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, 0)))
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4))
        model.compile(Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False), "mse")
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.0055 | Win/lose count 1.0/3.0 (-2.0)
Epoch 001/100 | Loss 0.0044 | Win/lose count 5.0/6.0 (-1.0)
Epoch 002/100 | Loss 0.0025 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/100 | Loss 0.0048 | Win/lose count 2.0/5.0 (-3.0)
Epoch 004/100 | Loss 0.0011 | Win/lose count 3.0/4.0 (-1.0)
Epoch 005/100 | Loss 0.0019 | Win/lose count 2.0/3.0 (-1.0)
Epoch 006/100 | Loss 0.0018 | Win/lose count 3.5/2.0 (1.5)
Epoch 007/100 | Loss 0.0011 | Win/lose count 1.5/2.0 (-0.5)
Epoch 008/100 | Loss 0.0038 | Win/lose count 5.5/5.0 (0.5)
Epoch 009/100 | Loss 0.0476 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/100 | Loss 0.0023 | Win/lose count 1.5/0 (1.5)
Epoch 011/100 | Loss 0.0028 | Win/lose count 2.0/2.0 (0.0)
Epoch 012/100 | Loss 0.0023 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/100 | Loss 0.0058 | Win/lose count 7.0/5.0 (2.0)
Epoch 014/100 | Loss 0.0020 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/100 | Loss 0.0511 | Win/lose count 3.5/1.0 (2.5)
Epoch 016/100 | Loss 0.0062 | Win/lose count 10.5/4

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
from keras.layers import Conv1D, Conv2D

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(64, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(4))
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0027 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/100 | Loss 0.0075 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/100 | Loss 0.0032 | Win/lose count 10.5/8.0 (2.5)
Epoch 003/100 | Loss 0.0216 | Win/lose count 9.5/1.0 (8.5)
Epoch 004/100 | Loss 0.0012 | Win/lose count 8.0/2.0 (6.0)
Epoch 005/100 | Loss 0.0073 | Win/lose count 7.0/6.0 (1.0)
Epoch 006/100 | Loss 0.0014 | Win/lose count 8.0/5.0 (3.0)
Epoch 007/100 | Loss 0.0017 | Win/lose count 11.0/2.0 (9.0)
Epoch 008/100 | Loss 0.0033 | Win/lose count 12.0/4.0 (8.0)
Epoch 009/100 | Loss 0.0041 | Win/lose count 10.0/5.0 (5.0)
Epoch 010/100 | Loss 0.0030 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/100 | Loss 0.0024 | Win/lose count 13.0/2.0 (11.0)
Epoch 012/100 | Loss 0.0045 | Win/lose count 4.5/6.0 (-1.5)
Epoch 013/100 | Loss 0.0018 | Win/lose count 8.5/3.0 (5.5)
Epoch 014/100 | Loss 0.0025 | Win/lose count 9.0/2.0 (7.0)
Epoch 015/100 | Loss 0.0014 | Win/lose count 6.0/1.0 (5.0)
Epoch 016/100 | Loss 0.0017 | Win/lose count 12.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 4.5/3.0. Average score (1.5)
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.0/0. Average score (1.375)
Win/lose count 1.5/1.0. Average score (1.2)
Win/lose count 7.5/2.0. Average score (1.9166666666666667)
Win/lose count 1.5/2.0. Average score (1.5714285714285714)
Win/lose count 2.0/5.0. Average score (1.0)
Win/lose count 4.5/1.0. Average score (1.2777777777777777)
Win/lose count 5.0/3.0. Average score (1.35)
Win/lose count 1.5/0. Average score (1.3636363636363635)
Win/lose count 3.0/3.0. Average score (1.25)
Win/lose count 2.5/1.0. Average score (1.2692307692307692)
Win/lose count 2.5/4.0. Average score (1.0714285714285714)
Win/lose count 5.5/0. Average score (1.3666666666666667)
Win/lose count 3.5/2.0. Average score (1.375)
Win/lose count 4.0/1.0. Average score (1.4705882352941178)
Win/lose count 4.0/3.0. Average score (1.4444444444444444)
Win/lose count 3.0/1.0. Average score (1.4736842105263157)


Win/lose count 2.5/16.0. Average score (-3.2127659574468086)
Win/lose count 3.5/11.0. Average score (-3.3020833333333335)
Win/lose count 6.5/4.0. Average score (-3.183673469387755)
Win/lose count 5.5/4.0. Average score (-3.09)
Win/lose count 3.0/7.0. Average score (-3.107843137254902)
Win/lose count 2.0/6.0. Average score (-3.125)
Win/lose count 4.5/4.0. Average score (-3.056603773584906)
Win/lose count 4.0/11.0. Average score (-3.1296296296296298)
Win/lose count 5.0/8.0. Average score (-3.1272727272727274)
Win/lose count 3.5/7.0. Average score (-3.1339285714285716)
Win/lose count 2.5/9.0. Average score (-3.192982456140351)
Win/lose count 1.5/14.0. Average score (-3.353448275862069)
Win/lose count 3.5/10.0. Average score (-3.406779661016949)
Win/lose count 3.5/13.0. Average score (-3.5083333333333333)
Win/lose count 5.5/7.0. Average score (-3.4754098360655736)
Win/lose count 3.5/3.0. Average score (-3.411290322580645)
Win/lose count 4.5/10.0. Average score (-3.4444444444444446)
Win/los

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

The CNN tends to perform slightly better than the FC at every temperature. 
The main issue is that at some point the rat tends to get stuck going back and forth in a few cells and doesn't explore the map anymore, which makes him miss out on some potential cheese. The problem is that he has no memory of where he already went, which means he may just revisit the same cells over and over again.

Another situation where his strategy is (understandably) not optimal is when he is surrounded by poison, but there is more cheese to be eaten further on the map. In this situation, the rat stays inside of his poison cage, whereas it would be profitable for him to poison himself on purpose to get out and be able to eat the remaining cheese. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [24]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    epsilon = agent.epsilon
    decay = 0.99

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)  # reset epsilon at each epoch

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(agent.epsilon * decay)  # We use a decreasing epsilon
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
        
        
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        

        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
            
        reward = 0
        if train:
            reward = - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1

        self.t = self.t + 1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    
    
    


In [25]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0153 | Win/lose count 14.5/47.500000000000036 (-33.000000000000036)
Epoch 001/100 | Loss 0.0123 | Win/lose count 17.0/43.10000000000005 (-26.10000000000005)
Epoch 002/100 | Loss 0.0075 | Win/lose count 19.5/23.1 (-3.6000000000000014)
Epoch 003/100 | Loss 0.0199 | Win/lose count 9.0/43.5 (-34.5)
Epoch 004/100 | Loss 0.0074 | Win/lose count 16.0/22.799999999999997 (-6.799999999999997)
Epoch 005/100 | Loss 0.0139 | Win/lose count 20.0/16.699999999999985 (3.300000000000015)
Epoch 006/100 | Loss 0.0114 | Win/lose count 24.5/20.9 (3.6000000000000014)
Epoch 007/100 | Loss 0.0059 | Win/lose count 25.5/11.999999999999984 (13.500000000000016)
Epoch 008/100 | Loss 0.0254 | Win/lose count 20.5/32.89999999999999 (-12.399999999999991)
Epoch 009/100 | Loss 0.0080 | Win/lose count 23.5/18.899999999999995 (4.600000000000005)
Epoch 010/100 | Loss 0.0087 | Win/lose count 20.0/18.799999999999994 (1.2000000000000064)
Epoch 011/100 | Loss 0.0036 | Win/lose count 16.5/18.49999999999999

Epoch 093/100 | Loss 0.0004 | Win/lose count 20.5/19.90000000000001 (0.5999999999999908)
Epoch 094/100 | Loss 0.0008 | Win/lose count 18.0/21.90000000000002 (-3.90000000000002)
Epoch 095/100 | Loss 0.0004 | Win/lose count 24.0/22.099999999999987 (1.9000000000000128)
Epoch 096/100 | Loss 0.0013 | Win/lose count 23.5/13.799999999999983 (9.700000000000017)
Epoch 097/100 | Loss 0.0016 | Win/lose count 24.5/20.09999999999999 (4.400000000000009)
Epoch 098/100 | Loss 0.0012 | Win/lose count 18.5/24.500000000000007 (-6.000000000000007)
Epoch 099/100 | Loss 0.0015 | Win/lose count 18.5/22.799999999999986 (-4.2999999999999865)


In [26]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore100.mp4'))

Win/lose count 22.0/2.0. Average score (20.0)
Win/lose count 23.0/2.0. Average score (20.5)
Win/lose count 24.5/4.0. Average score (20.5)
Win/lose count 25.5/3.0. Average score (21.0)
Win/lose count 19.0/3.0. Average score (20.0)
Win/lose count 25.0/2.0. Average score (20.5)
Win/lose count 19.0/4.0. Average score (19.714285714285715)
Win/lose count 21.5/3.0. Average score (19.5625)
Win/lose count 28.5/4.0. Average score (20.11111111111111)
Win/lose count 29.5/1.0. Average score (20.95)
Win/lose count 24.5/4.0. Average score (20.90909090909091)
Win/lose count 20.5/9.0. Average score (20.125)
Win/lose count 20.5/6.0. Average score (19.692307692307693)
Win/lose count 24.0/1.0. Average score (19.928571428571427)
Win/lose count 23.5/3.0. Average score (19.966666666666665)
Win/lose count 19.0/4.0. Average score (19.65625)
Win/lose count 25.5/4.0. Average score (19.764705882352942)
Win/lose count 25.5/3.0. Average score (19.916666666666668)
Win/lose count 29.0/6.0. Average score (20.078947368

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***